In [ ]:
import numpy as np
import re
from dataclasses import dataclass
import cv2 as cv

In [ ]:
test_text = """p=0,4 v=3,-3
p=6,3 v=-1,-3
p=10,3 v=-1,2
p=2,0 v=2,-1
p=0,0 v=1,3
p=3,0 v=-2,-2
p=7,6 v=-1,-3
p=3,0 v=-1,-2
p=9,3 v=2,3
p=7,3 v=-1,2
p=2,4 v=2,-3
p=9,5 v=-3,-3"""

In [ ]:
with open("input.txt") as f:
    input_text = f.read()

In [ ]:
@dataclass
class Robot:
    x: int
    y: int
    dx: int
    dy: int
    max_x: int
    max_y: int
    
    def move(self, dt: int):
        self.x = (self.x + self.dx * dt) % self.max_x
        self.y = (self.y + self.dy * dt) % self.max_y
        
    def get_quad(self):
        if self.x == self.max_x // 2:
            return False
        if self.y == self.max_y // 2:
            return False
        return (self.x // ((self.max_x + 1) // 2), self.y // ((self.max_y + 1) // 2))

In [ ]:
def parse_input(text: str, is_test: bool):
    lines = text.split("\n")
    pattern = r"p=(.+),(.+) v=(.+),(.+)"
    robots = []
    if is_test:
        size = [11, 7]
    else:
        size = [101, 103]
    for line in lines:
        if not line:
            continue
        matches = [int(x) for x in re.findall(pattern, line)[0]]
        robots.append(Robot(*matches, *size))
    return robots

### Part one

In [ ]:
robots = parse_input(input_text, False)

robot: Robot
quads = np.zeros((2, 2))
for i, robot in enumerate(robots):
    robot.move(100)
    quads[robot.get_quad()] += 1
res = 1
for i in quads.ravel():
    if i > 0:
        res *= i
int(res)

### Part two

In [ ]:
robots = parse_input(input_text, False)

robot: Robot

for t in range(1, 10000):
    result_img = np.zeros((robots[0].max_x, robots[0].max_y))
    for i, robot in enumerate(robots):
        robot.move(1)
        result_img[robot.x, robot.y] = 1
    result_img_erode = cv.erode(result_img, np.ones((5, 5)))
    if result_img_erode.sum() > 0:
        cv.imwrite(f"img_{t}.png", result_img * 255)
        print(t)